In [19]:
import numpy as np
import matplotlib
from mido import Message, MetaMessage
import mido
import sys
import random
import os
import time
import math
pwd = "/home/zedroman/Documents/Sonia_Data/Songs/60 FREE Melodies (MIDI)/"
savepwd="/home/zedroman/Documents/Sonia_Data/melodies"
music = sorted([f for f in os.listdir(pwd) if f[len(f)-4:len(f)] == ".mid"])
sos = 0
eos = 1
message_type = {"note_on":1, "note_off":0}
type_message = {1:"note_on", 0:"note_off"}
tempo = 500000 * 5
hidden_size = 586
classes = 586

In [20]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device="cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
def create_random(music):
    randoms = []
    for i in range(len(music)):
        r = []
        for j in range(3):
            r.append([random.random() * 10 for i in range(hidden_size)])
        randoms.append(r)
    return randoms
random_starts = create_random(music)

In [22]:
# 0(sos) 1(eos) 2 - 129(note_on) 130 - 257(note_off) 258 - 385(velocity) 386-585 //386-395,396-405, 406-415, 416-425 (time_shift)

def encode_type(tp, num):
    if (tp == "sos"):
        return 0
    if (tp == "eos"):
        return eos
    if (tp == "note_on"):
        return num + 2
    if (tp == "note_off"):
        return num + 130
    if (tp == "velocity"):
        return num + 258
    if (tp == "time"):
        t = (num // 8) + 386
        if (t < 586):
            return t
        return 585
    return -1

def get_type(c):
    if(c == 0):
        return "sos", c
    if(c == 1):
        return "eos", c
    if(c > 1 and c <= 129):
        return "note_on", c - 2
    
    if(c >= 130 and c < 258):
        return "note_off", c - 130
    
    if(c>=258 and c < 386):
        return "velocity", c - 258
    
    if(c>=386 and c < 586):
        return "time", int((c - 386) * 8)
    
    print(c)
    return "wtf", -1
    


def midi_to_list(mid=None, pwd=None, tensor=True):
    if(mid==None):
          mid = mido.MidiFile(pwd)
    sample=[]
    velocity = -1
    for i, track in enumerate(mid.tracks):
        j = 0
        for msg in track:
            j += 1
            if (j > 1000):
                break
            if (msg.type == "note_off" or msg.type == "note_on"):
                if (msg.time != 0):
                    c= encode_type("time", msg.time)
                    if (c != -1):
                        sample.append(c)
#                         for i in c:
#                             sample.append(i)
                if (msg.velocity != velocity):
                    c = encode_type("velocity", msg.velocity)
                    if (c != -1): 
                        sample.append(c)
                    velocity = msg.velocity
                if (msg.type == "note_on"):
                    c = encode_type("note_on", msg.note)
                    if (c != -1): 
                        sample.append(c)
                if (msg.type == "note_off"):
                    c = encode_type("note_off", msg.note)
                    if (c != -1): 
                        sample.append(c)
    sample.append(eos)
    if (tensor):
        return torch.tensor(sample, device=device)
    else:
        return sample
#     return sorted(sample, key=lambda x: x[-1])

def check_midi(sample):
    note_on = {}
    dele = []
    for j,i in enumerate(sample):
#         print(i)
        tp, nt = get_type(i)
        if (tp == "note_on"):
            if(nt in note_on):
                dele.append(j);
            note_on[nt] = 1;
        if (tp == "note_off"):
            if(nt in note_on):
                del note_on[nt]
            else:
                dele.append(j)
    for i in range(len(dele) - 1, -1, -1):
        del(sample[dele[i]])
    for i in note_on:
        sample.append(encode_type("note_off", i))

def list_to_midi(sample):
    print("was: ",len(sample))
    check_midi(sample)
    print("now: ",len(sample))
    
    midi = mido.MidiFile(type=0)
    tr = mido.MidiTrack()
    midi.tracks.append(tr)
    tr.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    velocity = 0
    time = 0
    for i in sample:
        tp, nt = get_type(i)
        if (tp == "velocity"):
            velocity = nt
        if (tp == "time"):
            time += nt
        if (tp == "note_off" or tp == "note_on"):
#             print(nt, velocity, time)
            tr.append(Message(tp, note=nt, velocity=velocity, time=time, channel=0))
            time = 0
    return midi

In [23]:
# for file in (music):
#     midi_file = mido.MidiFile(pwd + file)
#     sample = midi_to_list(mid=midi_file, tensor=False)
#     print(sample)
#     midi = list_to_midi(sample)

In [24]:

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [25]:
max_len = 1000
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru1 = nn.GRU(classes, hidden_size)
        self.gru2 = nn.GRU(classes, hidden_size)
        self.gru3 = nn.GRU(classes, hidden_size)
        self.out = nn.Linear(classes, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
        output = F.relu(input)
#         print(output.shape, hidden[0].shape)
        output, hidden[0] = self.gru1(output, hidden[0])
        output = F.relu(output)
        output, hidden[1] = self.gru2(output, hidden[1])
        output = F.relu(output)
        output, hidden[2] = self.gru3(output, hidden[2])
        
        
        output = self.softmax(self.out(output[0]))
        return output, hidden

In [26]:
teacher_forcing_ratio = 1

def one_hot(c):
    x = torch.zeros(classes, device=device, dtype=torch.float)
    x[c] = 1;
    return x.view(1,1,-1)

def train(target_tensor, decoder, decoder_optimizer, criterion, max_length=max_len, num = -1):
    decoder_optimizer.zero_grad()
    target_length = target_tensor.size(0)

    loss = 0

#     decoder_input = torch.tensor([[sos]], device=device)
    decoder_input = one_hot(sos)
    decoder_hidden =   [torch.tensor(random_starts[num][0], device=device, dtype=torch.float).view(1,1,-1),
                        torch.tensor(random_starts[num][1], device=device, dtype=torch.float).view(1,1,-1),
                        torch.tensor(random_starts[num][2], device=device, dtype=torch.float).view(1,1,-1)]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
#             if di == max_len:
#                 break
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di:di+1])
            decoder_input = one_hot(target_tensor[di])  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di:di+1])
            if decoder_input.item() == eos:
                break

    loss.backward()

    decoder_optimizer.step()

    return loss.item() / target_length

In [27]:
def trainIters(decoder, n_iters,decoder_optimizer, print_every=len(music), epoch=0):
    last_loss = 100;
    learning_rate = 0.001
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = music[iter % (len(music))]
#         print(training_pair)
        midi_file = mido.MidiFile(pwd + training_pair)
        target_tensor = midi_to_list(mid=midi_file)
        loss = train(target_tensor,
                     decoder, decoder_optimizer, criterion, num = iter % (len(music)))
        print_loss_total += loss

        if iter % print_every == 0:
            if (print_loss_total / print_every > last_loss):
#                 learning_rate /= 10
                print("learning rate change :", learning_rate)
#                 decoder_optimizer = optim.Adam(decoder.parameters() , lr=learning_rate)
            last_loss = print_loss_total / print_every
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [32]:
learning_rate = 0.00001
# decoder = DecoderRNN(hidden_size, 586).to(device)
decoder_optimizer = optim.Adam(decoder.parameters() , lr=learning_rate)

In [46]:
trainIters(decoder, 10000, decoder_optimizer)

0m 41s (- 114m 40s) (60 0%) 1.4454
1m 23s (- 114m 1s) (120 1%) 1.4385
2m 4s (- 113m 21s) (180 1%) 1.4338
2m 46s (- 112m 39s) (240 2%) 1.4268
3m 27s (- 111m 57s) (300 3%) 1.4200
4m 9s (- 111m 15s) (360 3%) 1.4130
4m 50s (- 110m 34s) (420 4%) 1.4066
5m 32s (- 109m 54s) (480 4%) 1.4001
6m 14s (- 109m 13s) (540 5%) 1.3932
6m 55s (- 108m 32s) (600 6%) 1.3873
7m 37s (- 107m 52s) (660 6%) 1.3807
8m 18s (- 107m 10s) (720 7%) 1.3746
9m 0s (- 106m 27s) (780 7%) 1.3682
9m 41s (- 105m 46s) (840 8%) 1.3618
10m 24s (- 105m 18s) (900 9%) 1.3562
11m 8s (- 104m 52s) (960 9%) 1.3505
11m 51s (- 104m 27s) (1020 10%) 1.3452
12m 34s (- 103m 53s) (1080 10%) 1.3397
13m 18s (- 103m 24s) (1140 11%) 1.3346
14m 1s (- 102m 51s) (1200 12%) 1.3288
14m 44s (- 102m 14s) (1260 12%) 1.3225
15m 27s (- 101m 35s) (1320 13%) 1.3164
16m 10s (- 101m 2s) (1380 13%) 1.3106
16m 53s (- 100m 25s) (1440 14%) 1.3051


KeyboardInterrupt: 

In [ ]:
def evaluate(decoder,num=-1, max_length=max_len):
    with torch.no_grad():

        decoder_input = torch.tensor([[sos]], device=device)  # SOS
        decoder_hidden = []
        for j in range(3):
            decoder_hidden.append([random.random() * 10 for i in range(hidden_size)])
        decoder_hidden =   [torch.tensor(decoder_hidden[0], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[1], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[2], device=device).view(1,1,-1)]

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(2)
            topv = topv.max().item() - topv.cpu()[0].numpy()
            summ = topv.sum()
            chosen = np.random.choice(topi[0].cpu(), p= topv / summ)
            decoder_input = torch.tensor(chosen, device=device)
            if decoder_input == eos:
#                 decoded_words.append('<EOS>')
#                 break
                decoder_input = torch.tensor([[sos]], device=device)
            else:
                decoded_words.append(chosen)

#             decoder_input = topi.data.topk(1)[1].squeeze().detach()

        return decoded_words

In [ ]:
def generate(number, decoder):
    for i in range(number):
        new_music = evaluate(decoder)
        midi = list_to_midi(new_music)
        print(midi.length)
        midi.save(savepwd +"test" + str(i) + ".mid")
    return midi

In [ ]:

last = generate(10, decoder)
for track in last.tracks:
    for msg in track:
        print(msg)

In [ ]:

for i in new_music:
    print(get_type(i))

In [ ]:
def test():
    midi = mido.MidiFile("/home/zedroman/Documents/Sonia_Data/Songs/test_from.mid")
#     midi = mido.MidiFile()
#     tr = mido.MidiTrack()
#     midi.tracks.append(tr)
#     tr.append(Message("note_on", velocity = 10, time= 1111, channel=0))
    sample = midi_to_list(midi, tensor=False)
#     for i in sample:
#         print(i)
    midi = list_to_midi(sample)
    midi.save("/home/zedroman/Documents/Sonia_Data/Songs/test_to.mid")
tempo = 500000 * 5
test()

In [ ]:
midi = mido.MidiFile(pwd + "2.mid")
for i, track in enumerate(midi.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
# tmp = music[0]
# music[0] =music[2]
# music[2] = tmp

In [ ]:
print(music)

In [48]:
torch.save(decoder.state_dict(), '/home/zedroman/Documents/Sonia_Data/test_models4/decoder5')
import pickle
with open('/home/zedroman/Documents/Sonia_Data/test_models4/starts4', "wb") as fp:   #Pickling
    pickle.dump(random_starts, fp)

In [43]:
decoder2 = DecoderRNN(hidden_size, 586).to(device)
checkpoint = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/decoder5')
decoder2.load_state_dict(checkpoint)
# decoder = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/decoder4')
with open('/home/zedroman/Documents/Sonia_Data/test_models4/starts4', "rb") as fp:   # Unpickling
    random_starts2 = pickle.load(fp)

True
